# Tuberculosis Diagnosis using Transfer Learning
This notebook is written and executed by **Dr Raheel Siddiqi** on *06-10-2019*. The notebook presents an experiment to classify X-ray images as 'NORMAL' or 'containing manifestation of Tuberculosis (TB)' i.e. it is a binary classification problem. Transfer Learning is used to exploit the feature extractor of the *VGG16* pre-trained model. The dataset used is: [**China Set - The Shenzhen set - Chest X-ray Database**](https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities). 10-fold cross validation is used to evaluate the model.

In [1]:
import tensorflow as tf
from tensorflow.python import keras

print('Tensorflow Version: ', tf.__version__)
print('Keras Version: ', keras.__version__)

Tensorflow Version:  1.13.1
Keras Version:  2.2.4-tf


In [2]:
from tensorflow.python.keras.applications import VGG16
import os
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import optimizers

def get_model():
    model = models.Sequential()
    conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(100,100,3))
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['accuracy'])
    return model   

In [3]:
image_height = 100
image_width = 100
batch_size = 2
no_of_epochs  = 15

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
%matplotlib inline

DATADIR = "D:\\TensorFlow Programs\\TB Diagnosis\\ChinaSet_AllFiles\\ChinaSet_AllFiles\\ChinaSetImages"

CATEGORIES = ["NORMAL", "TB"]

data=[]

for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)
    class_num=CATEGORIES.index(category)
    for img in tqdm(os.listdir(path)):  
        try:
            img_array = cv2.imread(os.path.join(path,img))
            img_array = cv2.resize(img_array, (image_height, image_width))
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)        
            img_array = img_array.astype(np.float32)/255.
            data.append([img_array, class_num])
        except Exception as e:   
            pass

print(len(data))

100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:32<00:00,  7.99it/s]


662


In [5]:
import random

random.shuffle(data)
for sample in data[:10]:
    print(sample[1])

1
1
0
0
1
0
0
0
1
1


In [6]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, image_width, image_height, 3)
print(X.shape)

(662, 100, 100, 3)


In [7]:
k=10
num_validation_samples=len(X)//k
validation_scores=[]
for fold in range(k):
    validation_data=X[num_validation_samples*fold:num_validation_samples*(fold+1)]
    validation_labels=y[num_validation_samples*fold:num_validation_samples*(fold+1)]
    if fold==0:
        training_data=X[num_validation_samples*(fold+1):]
        training_labels=y[num_validation_samples*(fold+1):]    
    else:
        training_data=np.append(X[:num_validation_samples*fold], X[num_validation_samples*(fold+1):],axis=0)
        training_labels=np.append(y[:num_validation_samples*fold], y[num_validation_samples*(fold+1):],axis=0)
    model=get_model()
    model.fit(training_data,training_labels,batch_size=4,epochs=5) # 5 epochs per model
    validation_score=model.evaluate(validation_data,validation_labels)
    validation_scores.append(validation_score[1])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
596/596 [==============================] - ETA: 6:43 - loss: 0.5817 - acc: 1.000 - ETA: 3:27 - loss: 0.6466 - acc: 0.875 - ETA: 2:21 - loss: 0.6164 - acc: 0.833 - ETA: 1:49 - loss: 0.5978 - acc: 0.812 - ETA: 1:29 - loss: 0.6342 - acc: 0.750 - ETA: 1:16 - loss: 0.6437 - acc: 0.708 - ETA: 1:06 - loss: 0.6473 - acc: 0.678 - ETA: 59s - loss: 0.6680 - acc: 0.593 - ETA: 54s - loss: 0.6796 - acc: 0.55 - ETA: 49s - loss: 0.6835 - acc: 0.55 - ETA: 46s - loss: 0.6986 - acc: 0.50 - ETA: 42s - loss: 0.6924 - acc: 0.54 - ETA: 40s - loss: 0.6869 - acc: 0.55 - ETA: 38s - loss: 0.6990 - acc: 0.53 - ETA: 36s - loss: 0.6992 - acc: 0.53 - ETA: 34s - loss: 0.6996 - acc: 0.53 - ETA: 32s - loss: 0.6922 - acc: 0.55 - ETA: 31s - loss: 0.6922 - acc: 0.55 - ETA: 30s - loss: 0.6946 - acc: 0.53 - ETA: 29s - loss: 0.6852 - acc: 0.56 - ETA: 28s - loss: 0.6808 - acc: 0.57 - ETA: 27s - los

596/596 [==============================] - ETA: 13s - loss: 0.4496 - acc: 0.75 - ETA: 13s - loss: 0.5482 - acc: 0.75 - ETA: 13s - loss: 0.5676 - acc: 0.75 - ETA: 13s - loss: 0.5494 - acc: 0.81 - ETA: 13s - loss: 0.5909 - acc: 0.75 - ETA: 13s - loss: 0.5961 - acc: 0.75 - ETA: 13s - loss: 0.6212 - acc: 0.75 - ETA: 13s - loss: 0.6125 - acc: 0.75 - ETA: 13s - loss: 0.6064 - acc: 0.77 - ETA: 12s - loss: 0.6103 - acc: 0.77 - ETA: 12s - loss: 0.6142 - acc: 0.77 - ETA: 12s - loss: 0.6016 - acc: 0.79 - ETA: 12s - loss: 0.6014 - acc: 0.78 - ETA: 12s - loss: 0.5907 - acc: 0.80 - ETA: 12s - loss: 0.5854 - acc: 0.81 - ETA: 12s - loss: 0.5871 - acc: 0.81 - ETA: 12s - loss: 0.5740 - acc: 0.82 - ETA: 12s - loss: 0.5745 - acc: 0.81 - ETA: 12s - loss: 0.5647 - acc: 0.82 - ETA: 12s - loss: 0.5496 - acc: 0.83 - ETA: 11s - loss: 0.5399 - acc: 0.84 - ETA: 11s - loss: 0.5337 - acc: 0.84 - ETA: 11s - loss: 0.5239 - acc: 0.84 - ETA: 11s - loss: 0.5112 - acc: 0.85 - ETA: 11s - loss: 0.5241 - acc: 0.83 - ETA: 11

596/596 [==============================] - ETA: 13s - loss: 0.9118 - acc: 0.50 - ETA: 13s - loss: 0.5290 - acc: 0.75 - ETA: 13s - loss: 0.5445 - acc: 0.75 - ETA: 13s - loss: 0.4521 - acc: 0.81 - ETA: 13s - loss: 0.5111 - acc: 0.75 - ETA: 13s - loss: 0.4832 - acc: 0.79 - ETA: 13s - loss: 0.4670 - acc: 0.78 - ETA: 13s - loss: 0.4332 - acc: 0.81 - ETA: 13s - loss: 0.3984 - acc: 0.83 - ETA: 12s - loss: 0.4167 - acc: 0.82 - ETA: 12s - loss: 0.4387 - acc: 0.81 - ETA: 12s - loss: 0.4326 - acc: 0.81 - ETA: 12s - loss: 0.4106 - acc: 0.82 - ETA: 12s - loss: 0.4592 - acc: 0.82 - ETA: 12s - loss: 0.4504 - acc: 0.81 - ETA: 12s - loss: 0.4439 - acc: 0.81 - ETA: 12s - loss: 0.4388 - acc: 0.80 - ETA: 12s - loss: 0.4346 - acc: 0.80 - ETA: 12s - loss: 0.4229 - acc: 0.80 - ETA: 12s - loss: 0.4151 - acc: 0.81 - ETA: 11s - loss: 0.4182 - acc: 0.80 - ETA: 11s - loss: 0.4664 - acc: 0.79 - ETA: 11s - loss: 0.4573 - acc: 0.79 - ETA: 11s - loss: 0.4657 - acc: 0.78 - ETA: 11s - loss: 0.4867 - acc: 0.77 - ETA: 11

596/596 [==============================] - ETA: 13s - loss: 0.6540 - acc: 0.75 - ETA: 13s - loss: 0.6039 - acc: 0.75 - ETA: 13s - loss: 0.4846 - acc: 0.83 - ETA: 13s - loss: 0.5839 - acc: 0.75 - ETA: 13s - loss: 0.5697 - acc: 0.75 - ETA: 13s - loss: 0.5221 - acc: 0.79 - ETA: 13s - loss: 0.5190 - acc: 0.78 - ETA: 13s - loss: 0.5806 - acc: 0.75 - ETA: 13s - loss: 0.6062 - acc: 0.72 - ETA: 12s - loss: 0.5710 - acc: 0.75 - ETA: 12s - loss: 0.5435 - acc: 0.77 - ETA: 12s - loss: 0.5389 - acc: 0.77 - ETA: 12s - loss: 0.5393 - acc: 0.76 - ETA: 12s - loss: 0.5361 - acc: 0.76 - ETA: 12s - loss: 0.5568 - acc: 0.75 - ETA: 12s - loss: 0.5513 - acc: 0.75 - ETA: 12s - loss: 0.5383 - acc: 0.75 - ETA: 12s - loss: 0.5298 - acc: 0.75 - ETA: 12s - loss: 0.5487 - acc: 0.75 - ETA: 12s - loss: 0.5469 - acc: 0.76 - ETA: 11s - loss: 0.5602 - acc: 0.76 - ETA: 11s - loss: 0.5681 - acc: 0.75 - ETA: 11s - loss: 0.6010 - acc: 0.72 - ETA: 11s - loss: 0.5986 - acc: 0.72 - ETA: 11s - loss: 0.5847 - acc: 0.74 - ETA: 11

66/66 [==============================] - ETA: 0s - loss: 0.4488 - acc: 0.843 - ETA: 0s - loss: 0.4683 - acc: 0.828 - 1s 19ms/sample - loss: 0.4615 - acc: 0.8333
Epoch 1/5


...

596/596 [==============================] - ETA: 2:05 - loss: 0.6205 - acc: 0.750 - ETA: 1:09 - loss: 0.6260 - acc: 0.750 - ETA: 50s - loss: 0.7544 - acc: 0.583 - ETA: 41s - loss: 0.7364 - acc: 0.62 - ETA: 35s - loss: 0.6999 - acc: 0.70 - ETA: 31s - loss: 0.7236 - acc: 0.66 - ETA: 28s - loss: 0.7979 - acc: 0.60 - ETA: 26s - loss: 0.8361 - acc: 0.53 - ETA: 25s - loss: 0.8199 - acc: 0.52 - ETA: 23s - loss: 0.8036 - acc: 0.55 - ETA: 22s - loss: 0.7946 - acc: 0.54 - ETA: 21s - loss: 0.7886 - acc: 0.54 - ETA: 20s - loss: 0.7735 - acc: 0.55 - ETA: 20s - loss: 0.7597 - acc: 0.57 - ETA: 19s - loss: 0.7349 - acc: 0.60 - ETA: 18s - loss: 0.7389 - acc: 0.59 - ETA: 18s - loss: 0.7282 - acc: 0.60 - ETA: 17s - loss: 0.7187 - acc: 0.61 - ETA: 17s - loss: 0.7256 - acc: 0.60 - ETA: 17s - loss: 0.7436 - acc: 0.58 - ETA: 16s - loss: 0.7348 - acc: 0.59 - ETA: 16s - loss: 0.7423 - acc: 0.57 - ETA: 16s - loss: 0.7458 - acc: 0.56 - ETA: 15s - loss: 0.7450 - acc: 0.55 - ETA: 15s - loss: 0.7429 - acc: 0.55 - ET

596/596 [==============================] - ETA: 13s - loss: 0.6767 - acc: 1.00 - ETA: 13s - loss: 0.6881 - acc: 0.62 - ETA: 13s - loss: 0.6850 - acc: 0.66 - ETA: 13s - loss: 0.6804 - acc: 0.68 - ETA: 13s - loss: 0.6880 - acc: 0.60 - ETA: 13s - loss: 0.6878 - acc: 0.58 - ETA: 13s - loss: 0.6785 - acc: 0.60 - ETA: 13s - loss: 0.6711 - acc: 0.62 - ETA: 13s - loss: 0.6676 - acc: 0.63 - ETA: 13s - loss: 0.6634 - acc: 0.65 - ETA: 13s - loss: 0.6617 - acc: 0.65 - ETA: 12s - loss: 0.6441 - acc: 0.68 - ETA: 12s - loss: 0.6506 - acc: 0.67 - ETA: 12s - loss: 0.6296 - acc: 0.69 - ETA: 12s - loss: 0.6387 - acc: 0.68 - ETA: 12s - loss: 0.6341 - acc: 0.68 - ETA: 12s - loss: 0.6443 - acc: 0.67 - ETA: 12s - loss: 0.6531 - acc: 0.66 - ETA: 12s - loss: 0.6478 - acc: 0.67 - ETA: 12s - loss: 0.6693 - acc: 0.63 - ETA: 12s - loss: 0.6694 - acc: 0.63 - ETA: 12s - loss: 0.6690 - acc: 0.63 - ETA: 11s - loss: 0.6623 - acc: 0.65 - ETA: 11s - loss: 0.6506 - acc: 0.66 - ETA: 11s - loss: 0.6481 - acc: 0.67 - ETA: 11

596/596 [==============================] - ETA: 13s - loss: 0.3846 - acc: 1.00 - ETA: 13s - loss: 0.4863 - acc: 0.87 - ETA: 13s - loss: 0.5138 - acc: 0.83 - ETA: 13s - loss: 0.4737 - acc: 0.81 - ETA: 13s - loss: 0.6022 - acc: 0.75 - ETA: 13s - loss: 0.6344 - acc: 0.75 - ETA: 13s - loss: 0.7119 - acc: 0.67 - ETA: 13s - loss: 0.6711 - acc: 0.71 - ETA: 13s - loss: 0.6269 - acc: 0.75 - ETA: 13s - loss: 0.6013 - acc: 0.77 - ETA: 13s - loss: 0.6235 - acc: 0.72 - ETA: 12s - loss: 0.6035 - acc: 0.72 - ETA: 12s - loss: 0.5953 - acc: 0.73 - ETA: 12s - loss: 0.5771 - acc: 0.75 - ETA: 12s - loss: 0.5917 - acc: 0.75 - ETA: 12s - loss: 0.5884 - acc: 0.75 - ETA: 12s - loss: 0.5717 - acc: 0.76 - ETA: 12s - loss: 0.5863 - acc: 0.75 - ETA: 12s - loss: 0.5883 - acc: 0.73 - ETA: 12s - loss: 0.5785 - acc: 0.73 - ETA: 12s - loss: 0.5683 - acc: 0.75 - ETA: 12s - loss: 0.5643 - acc: 0.75 - ETA: 11s - loss: 0.5809 - acc: 0.72 - ETA: 11s - loss: 0.5829 - acc: 0.71 - ETA: 11s - loss: 0.5727 - acc: 0.73 - ETA: 11

596/596 [==============================] - ETA: 13s - loss: 0.2681 - acc: 1.00 - ETA: 13s - loss: 0.2625 - acc: 1.00 - ETA: 13s - loss: 0.3985 - acc: 0.83 - ETA: 13s - loss: 0.4573 - acc: 0.81 - ETA: 13s - loss: 0.4019 - acc: 0.85 - ETA: 13s - loss: 0.4366 - acc: 0.83 - ETA: 13s - loss: 0.4585 - acc: 0.82 - ETA: 13s - loss: 0.5412 - acc: 0.78 - ETA: 13s - loss: 0.5302 - acc: 0.77 - ETA: 13s - loss: 0.5389 - acc: 0.75 - ETA: 13s - loss: 0.5422 - acc: 0.75 - ETA: 12s - loss: 0.5082 - acc: 0.77 - ETA: 12s - loss: 0.4980 - acc: 0.76 - ETA: 12s - loss: 0.4902 - acc: 0.76 - ETA: 12s - loss: 0.4715 - acc: 0.78 - ETA: 12s - loss: 0.4662 - acc: 0.78 - ETA: 12s - loss: 0.4879 - acc: 0.77 - ETA: 12s - loss: 0.4739 - acc: 0.79 - ETA: 12s - loss: 0.4560 - acc: 0.80 - ETA: 12s - loss: 0.4693 - acc: 0.78 - ETA: 12s - loss: 0.4622 - acc: 0.79 - ETA: 12s - loss: 0.4566 - acc: 0.80 - ETA: 11s - loss: 0.4536 - acc: 0.80 - ETA: 11s - loss: 0.4459 - acc: 0.81 - ETA: 11s - loss: 0.4313 - acc: 0.82 - ETA: 11

66/66 [==============================] - ETA: 0s - loss: 0.3924 - acc: 0.875 - ETA: 0s - loss: 0.4149 - acc: 0.843 - 1s 8ms/sample - loss: 0.4301 - acc: 0.8333


In [8]:
print('Average Validation Score: ', np.average(validation_scores))

Average Validation Score:  0.8
